In [4]:
# Kill all GPU processes to free memory
# !kill -9 $(nvidia-smi --query-compute-apps=pid --format=csv,noheader) 2>/dev/null || true
# !nvidia-smi

In [ ]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from transformers import TrainingArguments, Trainer, BitsAndBytesConfig
import peft

import os
from huggingface_hub import login
import gc

# Access Llama model
token = # insert token #
login(token)

: 

In [16]:
# Clear GPU memory and restart Python kernel state
torch.cuda.empty_cache()
gc.collect()

# Check available memory
print(f"GPU memory allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
print(f"GPU memory reserved: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")

GPU memory allocated: 0.01 GB
GPU memory reserved: 0.02 GB


In [19]:
# Load model with unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
    device_map={"": 0},  # Force everything to GPU 0
)

print(f"Model loaded! GPU memory now: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")

==((====))==  Unsloth 2025.12.1: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


==((====))==  Unsloth 2025.12.1: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


OutOfMemoryError: CUDA out of memory. Tried to allocate 28.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 28.12 MiB is free. Process 64797 has 11.02 GiB memory in use. Process 140169 has 3.69 GiB memory in use. Of the allocated memory 3.55 GiB is allocated by PyTorch, and 10.04 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Test simple inference
messages = [
    {"role": "user", "content": "The capital of France is"},
]
output = pipe(messages, max_new_tokens=50)
print(output[0]['generated_text'][-1]['content'])

In [ ]:
from peft import LoraConfig, get_peft_model
